In my down time, I've been writing documentation for [Sampyl](https://github.com/mcleonard/sampyl), a necessary and sometimes fun task. I built the documentation with [Sphinx](http://sphinx-doc.org/), a very nice package that allows you to focus on the content. Then, I wanted to find somewhere to host the documentation online for free. My first attempt was with [Read the Docs](https://readthedocs.org/). After running into some problems with Read the Docs building my documentation, I tried hosting on GitHub. That didn't work immediately either, but after fixing the issues, I wanted to share my experiences as to guide others.

<!-- fold -->

For Read the Docs, you point it to your GitHub repository which should contain a `docs` folder with the Sphinx `conf.py` file and all the other files necessary for building the documentation. Then, the documentation is built automatically and hosted on Read the Docs. It's all pretty simple, at least if you aren't using any packages with C extensions. Unfortunately, Sampyl relies on Numpy quite heavily, and Scipy in some places. You can deal with this by [mocking the packages](https://docs.readthedocs.org/en/latest/faq.html#i-get-import-errors-on-libraries-that-depend-on-c-modules), but then this broke some of my code. For instance, when parameters are bounded for some $P(X)$, then $\log{P(X)} = -\infty$ if $X$ is out of bounds. To deal with this, we write $\log{P(X)}$ functions like this:

In [ ]:
def logp(X):
    if outofbounds(X):
        return -np.inf
    ....

Now, this looks fine, until you try to build it when Numpy is mocked. Numpy understands how to deal with that negative sign, but it is undefined for most objects (try `a = -[1,2,3]`). When I mocked Numpy, the build system couldn't handle the negative sign because `np` wasn't Numpy any more. I changed it to `np.negative(np.inf)` even though I really didn't want to change the code to get the documentation to build. With that error solved, I ran into another problem with Scipy. At that point, I didn't want to keep struggling just to host some HTML files. I was using Sphinx to generate all the HTML, CSS, and JaveScript files myself, I just needed somewhere to host the files. I rolled back my changes and looked for another option.

This is when I checked out GitHub Pages. It is pretty straightforward to create a [GitHub project page](https://help.github.com/articles/creating-project-pages-manually/) from your own files. I won't go into that here, because their instructions are clear and correct. However, after following the instructions, I checked out the documentation on GitHub and found that the CSS files weren't being loaded. After some searching online, here's what happened. GitHub uses [Jekyll](http://jekyllrb.com/) to build its pages. Jekyll ignores folders that start with an underscore, like `_static`. By default, Sphinx stores CSS files in the folder `_static`. The solution is to include an empty file in the documentation repository named `.nojekyll`, telling GitHub to bypass Jekyll. Alternatively, you can set the name of the static folder to `static` in `conf.py`, but I think adding `.nojekyll` will take care of any issues all at once.

So finally, I got the documentation online. It is still sparse, but you can check it out [here](http://mcleonard.github.io/sampyl/).